## Test trained U-Net
#### Use HCC-trained U-Net on biopsies

In [1]:
import os
from random import sample, seed
import cv2
import numpy as np
from keras.models import load_model
import csv

Using TensorFlow backend.


In [2]:
slide_name="R10_004_1- 2017-08-25"

In [3]:
path="E:\\deeplearning\\Hepatocarcinomes\\data\\5x\\biopsy_HCC"
path1=path+'\\'+slide_name
path=path1+"\\512_image_label"
path2=path1+"\\results"

In [4]:
limg = []
for root, dirs, files in os.walk(path):
    for f in files:
        if f.endswith('.tif'):
            limg.append(root + '\\' + f)

In [5]:
img = np.ndarray((len(limg)*64, 192, 192))

In [6]:
for i in range(len(limg)):
    image = cv2.imread(limg[i], 0)/255
    reflect = cv2.copyMakeBorder(image,64,64,64,64,cv2.BORDER_REFLECT)
    for j in range(8):
        for k in range(8):
            img[i*64+j*8+k,:,:] = reflect[k*64:k*64+192,j*64:j*64+192]

In [7]:
model = load_model('E:\\deeplearning\\Hepatocarcinomes\\models\\5x\\unet\\2_best_model.hdf5')

In [8]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 64) 640         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 256, 256, 64) 36928       conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 128, 128, 64) 0           conv2d_2[0][0]                   
__________________________________________________________________________________________________
conv2d_3 (

In [9]:
def testGenerator(img,target_size = (256,256),flag_multi_class = False):
    for i in range(img.shape[0]):
        image = cv2.resize(img[i],target_size) #resize function only support float image
#         print(img.shape)
        image = np.reshape(image,image.shape+(1,)) if (not flag_multi_class) else image
        image = np.reshape(image,(1,)+image.shape)
#         print(img.shape)
        yield image
    
preds = model.predict_generator(testGenerator(img),img.shape[0],verbose=1)

33600/33600 [==============================] - 884s 26ms/step


In [10]:
# with open(path2+'\\vgg_dense_bn_prob.csv', 'w', newline='') as csvfile:
#     writer = csv.writer(csvfile, delimiter=',')
#     writer.writerow(['tum','nor'])
#     writer.writerows(preds)
#     for l in preds:
#         writer.writerow(l)

In [16]:
mask = np.ndarray((512, 512))
for i,item in enumerate(preds):
    res = item[:,:,0]
    large = cv2.resize(res, (192, 192))
    mask[i%64%8*64:(i%64%8+1)*64,i%64//8*64:(i%64//8+1)*64] = large[64:128,64:128]
    if (i%64 == 63):
        n = str(len(os.listdir(path2+'\\label_overlap')))
        cv2.imwrite(path2+'\\label_overlap\\'+n.zfill(5)+'.tiff',mask*255)

In [12]:
# fpr, tpr, thresholds  =  roc_curve(y_test, preds[:,0])
# roc_auc = auc(fpr, tpr)
# # worse example (use the line y=1-x)
# # for i in range(len(fpr)):
# #     if fpr[i] + tpr[i] >= 1:
# #         i = i -1
# #         break

# # Normally, we use Youden's J statistic.
# i = np.argmax(tpr - fpr)

# # plt.plot(fpr,tpr) to draw. 'roc_auc' records the value of auc, which can be calculated by the auc() function.
# plt.plot(fpr, tpr, lw=1, label='auc = %0.3f, tpr = %0.3f, tnr = %0.3f, thres = %0.3f' % ( roc_auc, tpr[i], 1-fpr[i], thresholds[i]))
# # Draw diagonal
# plt.plot([0, 1], [0, 1], '--', color=(0.6, 0.6, 0.6))
# # Draw perfect preformance
# # plt.plot([0, 0, 1], [0, 1, 1],
# #          linestyle = ':',
# #          color = 'black')

# plt.xlim([-0.05, 1.05])
# plt.ylim([-0.05, 1.05])
# plt.xlabel('False Positive Rate (1 - Specificity)')
# plt.ylabel('True Positive Rate (Sensitivity)')
# plt.title('ROC curve of zeiss test set')
# plt.legend(loc="lower right")
# #     plt.show()
# plt.savefig(path2 + '//ROC.jpg')
# plt. close() 

In [13]:
# post = np.ndarray((512, 512),dtype='uint')
# for i,item in enumerate(preds):
#     res = item[:,:,0]
#     post[i%64%8*64:(i%64%8+1)*64,i%64//8*64:(i%64//8+1)*64] = cv2.resize(res, (64, 64))
#     if (i%64 == 63):
#         n = str(len(os.listdir(path2+'\\post')))
#         blur = cv2.GaussianBlur(post,(5,5),0)*255
# #         ret,post = cv2.threshold(blur,100,255,cv2.THRESH_TOZERO)
#         for i in range(512):
#             for j in range(512):
#                 f = lambda x: 1 if x<=100 else 2 if x<=200 else 3
#                 blur[i, j] = f(blur[i, j])
#         cv2.imwrite(path2+'\\post\\'+n.zfill(5)+'.tiff',blur)

In [15]:
path2


'E:\\deeplearning\\Hepatocarcinomes\\data\\5x\\biopsy_HCC\\R10_004_1- 2017-08-25\\results'